# Install Fabric Utils and Import Libraries

In [ ]:
%run "/Cricsheet Initialize"

# Set Parameters

In [ ]:
semantic_model_list = "Cricsheet Model" # Can pass multiple coma seperated "model1,model2"
master_job_name = "Cricsheet Master"
optimize_and_vacuum = False # Change to True to enable optimize and vaccum
items_to_optimize_vacuum = {CLEAN_LAKEHOUSE: None, RAW_LAKEHOUSE: None} # None corresponds to all tables in Lakehouse

# Create DAG

In [ ]:
DAG = {
    "activities": [
        {
            "name": "Cricsheet Ingest Data", # activity name, must be unique
            "path": "Cricsheet Ingest Data", # notebook path
            "timeoutPerCellInSeconds": 1800, # max timeout for each cell, default to 90 seconds
            "args": {"master_job_name": master_job_name} # notebook parameters
        },
        {
            "name": "Cricsheet Build Facts and Dimensions",
            "path": "Cricsheet Build Facts and Dimensions",
            "timeoutPerCellInSeconds": 1800,
            "dependencies": ["Cricsheet Ingest Data"],
            "args": {"master_job_name": master_job_name}
        },
        {
            "name": "Cricsheet Optimize and Vacuum",
            "path": "Cricsheet Optimize and Vacuum",
            "timeoutPerCellInSeconds": 1800,
            "dependencies": ["Cricsheet Build Facts and Dimensions"],
            "args": {"optimize_and_vacuum": optimize_and_vacuum, "master_job_name": master_job_name, "items_to_optimize_vacuum": str(items_to_optimize_vacuum)}
        },
        {
            "name": "Cricsheet Model Refresh",
            "path": "Cricsheet Model Refresh",
            "timeoutPerCellInSeconds": 180,
            "dependencies": ["Cricsheet Optimize and Vacuum"],
            "args": {"semantic_model_list_str": semantic_model_list, "master_job_name": master_job_name}
        }
    ]
}

# Execute DAG

In [ ]:
U.execute_and_log(function=U.execute_dag,
                log_lakehouse=RAW_LAKEHOUSE, 
                job_name=master_job_name,
                job_category= "Orchestrator",
                dag=DAG)